# Chat API LLM Detector - Google Colab

Detect which chat API (ChatGPT, Claude, Grok, Gemini, or DeepSeek) generated a piece of text.

**Important:** Make sure to enable GPU runtime!
- Go to: Runtime → Change runtime type → Hardware accelerator → GPU (T4)

## Step 1: Install Dependencies

In [ ]:
!pip install -q llm2vec==0.2.3 huggingface-hub

## Step 2: Download the Classifier

This will download ~16GB. It may take 5-10 minutes.

In [ ]:
!huggingface-cli download Yida/classifier_chat --local-dir ./classifier_chat

## Step 3: Load the Classifier

In [ ]:
import torch
import numpy as np
from transformers import AutoConfig, AutoModel, AutoTokenizer
from peft import PeftModel
from llm2vec import LLM2Vec

def load_classifier(checkpoint_path="./classifier_chat", num_labels=5):
    """Load the pre-trained LLM2Vec classifier."""
    print("Loading classifier...")
    
    base_model_name = "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp"
    
    config = AutoConfig.from_pretrained(
        base_model_name,
        trust_remote_code=True,
    )
    model = AutoModel.from_pretrained(
        base_model_name,
        config=config,
        torch_dtype=torch.bfloat16,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        trust_remote_code=True,
    )
    
    model = PeftModel.from_pretrained(
        model,
        base_model_name,
        torch_dtype=torch.bfloat16,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        trust_remote_code=True,
    )
    model = model.merge_and_unload()
    
    model = PeftModel.from_pretrained(
        model,
        f"{base_model_name}-supervised",
        is_trainable=True,
        torch_dtype=torch.bfloat16,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        trust_remote_code=True,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
    model = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)
    
    hidden_size = list(model.modules())[-1].weight.shape[0]
    model.head = torch.nn.Linear(hidden_size, num_labels, dtype=torch.bfloat16)
    
    import os
    head_path = os.path.join(checkpoint_path, "head.pt")
    model.head.load_state_dict(torch.load(head_path, map_location="cuda" if torch.cuda.is_available() else "cpu"))
    
    model.eval()
    print(f"✓ Classifier loaded successfully! Using: {'GPU' if torch.cuda.is_available() else 'CPU'}")
    return model

# Load the model
model = load_classifier()

## Step 4: Define Prediction Function

In [ ]:
def predict_text(model, text):
    """Predict which LLM generated the given text."""
    label_names = ["ChatGPT", "Claude", "Grok", "Gemini", "DeepSeek"]
    
    # Prepare text
    prepared_text = model.prepare_for_tokenization(text)
    inputs = model.tokenize([prepared_text])
    
    # Move to device
    device = next(model.parameters()).device
    inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in inputs.items()}
    
    # Predict
    with torch.no_grad():
        embeddings = model.forward(inputs).to(torch.bfloat16)
        logits = model.head(embeddings)
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    pred_label = torch.argmax(probabilities, dim=-1).item()
    all_probs = probabilities[0].cpu().numpy()
    
    # Display results
    print("\n" + "="*60)
    print("PREDICTION RESULTS")
    print("="*60)
    print(f"\nMost likely source: {label_names[pred_label]}")
    print(f"Confidence: {all_probs[pred_label]*100:.2f}%")
    print("\nAll probabilities:")
    print("-"*60)
    
    sorted_indices = np.argsort(all_probs)[::-1]
    for idx in sorted_indices:
        bar_length = int(all_probs[idx] * 50)
        bar = "█" * bar_length
        print(f"{label_names[idx]:20s} {all_probs[idx]*100:6.2f}% {bar}")
    print("="*60)
    
    return label_names[pred_label], all_probs[pred_label]

## Step 5: Test with Sample Text

Replace the text below with any text you want to classify:

In [ ]:
# Example text - replace with your own!
sample_text = """
Hello! I'd be happy to help you with that question. Let me break this down into a few key points:

1. First, it's important to understand the context
2. Second, we should consider the implications
3. Finally, let's look at practical applications

I hope this helps clarify things for you!
"""

predict_text(model, sample_text)

## Step 6: Classify Your Own Text

Paste your text in the input box below:

In [ ]:
# Interactive input
print("Paste your text below and press Enter:")
user_text = input()

if user_text.strip():
    predict_text(model, user_text)
else:
    print("No text provided!")

## Multiple Texts

You can test multiple texts at once:

In [ ]:
# Test multiple texts
texts_to_test = [
    "Sure, I can help with that!",
    "I'd be happy to assist you with this question.",
    "Let me break this down for you step by step.",
]

for i, text in enumerate(texts_to_test, 1):
    print(f"\n{'='*60}")
    print(f"TEXT #{i}: {text[:50]}...")
    predict_text(model, text)